In [1]:
from bs4 import BeautifulSoup
import urllib.request
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [100]:
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np

In [3]:
# scrap dari digilib

In [4]:
docs_num = 20

In [5]:
digilib_url = 'http://digilib.its.ac.id/publisher/51100/'

In [6]:
paper = []

In [7]:
for i in range(0,docs_num,20):
    page = urllib.request.urlopen(digilib_url)
    soup = BeautifulSoup(page, 'html.parser')
    docs = soup.find_all('span', attrs={'class': 'style5'})
    link = []
    for x in docs:
        try:
            link.append(x.find('a').get('href'))
        except:
            pass
    for x in tqdm(link[:20], desc='paper', unit='paper'):
        page = urllib.request.urlopen(x)
        soup = BeautifulSoup(page, 'html.parser')
        try:
            title = soup.find('h2', attrs={'class': 'isi'}).find('i').getText()
            abstract = soup.find('span', attrs={'class': 'teks'}).find('p').getText()
            paper.append([x, title, abstract])
        except:
            pass

paper: 100%|████████████████████████████████| 20/20 [00:15<00:00,  1.29paper/s]


In [8]:
len(paper)

18

In [9]:
# preprocessing

In [10]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
stemmer = StemmerFactory().create_stemmer()

In [12]:
words = []
processed_paper = [x[:2] for x in paper]

In [14]:
for num, x in enumerate(paper):
    text = x[2]
    text = text.lower()
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    text = text.translate(remove_punctuation_map)
    text = stopword.remove(text)
    text = text.split()
    text = [stemmer.stem(x) for x in text]
    processed_paper[num].append(' '.join(text))
    text = list(set(text))
    words += text

In [15]:
processed_paper[:5]

[['http://digilib.its.ac.id/ITS-paper-51021160009033/41690',
  'RANCANG BANGUN APLIKASI KOMUNIKASI AUDIO PADA JARINGAN NIRKABEL LOKAL BERBASIS ANDROID MENGGUNAKAN ALGORITMA JOINT CODING RATE CONTROL',
  'kembang aplikasi android dewasa sangat pesat aplikasi kembang selesai masalah bagai bidang beberapa kategori aplikasi android kembang game aplikasi media sosial bahkan aplikasi ganti komunikasi telepon aplikasi kirim pesan komunikasi audio video aplikasi komunikasi audio rupa salah satu popular seringkali kualitas kualitas kirim audio turun iring turun kualitas jaring sebab komunikasi jadi kurang nyaman smartphone android banyak lengkap konektivitas bluetooth wifi smartphone android mulai versi 40 sudah dukung koneksi langsung antar peers juga lowlevel audio api akses driver suara perangkat tugas akhir ini usul solusi permasalahandengan manfaat teknologi android buat aplikasi audio bisa guna komunikasi guna infrastruktur sudah maupun guna koneksi langsung antar peers tingkat kualitas a

In [ ]:
# tf_idf

In [16]:
processed_abstract = [x[2] for x in processed_paper]

In [92]:
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, use_idf=True)
query = 'hosting aplikasi internet'
query = [query]
paper_tfidf = vectorizer.fit_transform(query + processed_abstract) 

In [ ]:
# result

In [96]:
query = paper_tfidf[0]
result = cosine_similarity(paper_tfidf, query)

In [116]:
idx = np.argsort(-result,axis=0)[:6].flatten()[1:]
for x in idx:
    print('Paper', x, 'score :', result[x][0])

Paper 3 score : 0.4048475908081165
Paper 4 score : 0.4048475908081165
Paper 1 score : 0.14457989044139255
Paper 2 score : 0.14457989044139255
Paper 18 score : 0.044259448151226566


In [121]:
for x in range(len(idx)): 
    print('Result', x+1) 
    print(processed_paper[idx[x]-1][1]) 
    print(processed_paper[idx[x]-1][2])
    print()

Result 1
RANCANG BANGUN LAYANAN PLATFORM AS A SERVICE PAAS UNTUK MENDUKUNG SISTEM MULTI-TENANCY PENGEMBANGAN APLIKASI BERBASIS KOMPUTASI AWAN
layan hosting aplikasi sering sebut application hosting web hosting rupa layan jamak sedia indonesia layan beri wadah siapa menempatakan aplikasi web serta basis data jaring internet perlu sedia sendiri infrastruktur internet dedikasi layan milik bagai macam bentuk salah satu paling banyak sedia indonesia shared web hosting sedia layan sangat murah jenis layan sewa beri satu buah platform sama aplikasi simpan sama satu buah server mungkin ada bottleneck itu dapat jenis layan virtual private server vps sedia kendali penuh sistem lalu teknik sebut virtual konfigurasi sangat kompleks dapat jenis layan hosting baru sebut cloud web hosting layan adopsi prinsip komputasi awan upa platformasaservice paas sedia jasa platform kembang web tempat aplikasi internet layan tawar fitur transparansi akses multiplatform skalabilitas reliabilitas buka api mudah gu